In [1]:
### plotting
import matplotlib.pyplot as plt

### user module
from model.transit_sim_model import Network, Trains, Travelers

### fix random seed
import numpy as np
np.random.seed(0)

### Process GTFS schedules
Need the following tables:
 * stop_times.txt: schedule info
 * trips.txt: map trip_id to route_id
 * stops.txt: get stop coordinates (for visualization)

In [2]:
### read in GTFS files
stop_times_file = 'example_bart_google_transit_20211101_20220213_v2/stop_times.txt'
trips_file = 'example_bart_google_transit_20211101_20220213_v2/trips.txt'
stops_file = 'example_bart_google_transit_20211101_20220213_v2/stops.txt'
### only keep results with this service id
service_id = '2021_09_13-DX-MVS-Weekday-08'

### create all trains from GTFS
all_trains = Trains()
all_nodes, all_links = all_trains.schedule_and_network_from_gtfs(
    stop_times_file, trips_file, stops_file, service_id)

### create network from nodes and links
network = Network(all_nodes, all_links)

/Users/bingyu/Desktop/transit_sim/model/transit_sim_model.py:156: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schedule_table['stop_lon'] = schedule_table.geometry.centroid.x
/Users/bingyu/Desktop/transit_sim/model/transit_sim_model.py:157: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schedule_table['stop_lat'] = schedule_table.geometry.centroid.y


In [3]:
### display and export network
display(network.all_nodes.head(3))
display(network.all_links.head(2))
network.all_links.to_csv('outputs/example_links.csv', index=False)
network.all_nodes.to_csv('outputs/example_nodes.csv', index=False)

,route_stop_id,stop_lon,stop_lat,stop_id,type,node_id,geometry
0,5-BERY,-121.874322,37.368759,BERY,platform,0,POINT (-121.87432 37.36876)
1,5-MLPT,-121.890722,37.410562,MLPT,platform,1,POINT (-121.89072 37.41056)
2,5-WARM,-121.939036,37.50257,WARM,platform,2,POINT (-121.93904 37.50257)


,route_stop_id,next_route_stop_id,start_nid,end_nid,initial_weight,geometry
66,5-BERY,5-MLPT,0,1,1.0,"LINESTRING (-121.87432 37.36876, -121.89072 37..."
67,5-MLPT,5-WARM,1,2,1.0,"LINESTRING (-121.89072 37.41056, -121.93904 37..."


### Travel demand
 * random demand
 * or, input csv with columns *traveler_id*, *origin_nid*, *destin_nid*, *departure_time*

In [9]:
### random OD
### create instance
travelers = Travelers()
### generate random od
travelers.random_od(network.all_nodes, num_travelers=100)
### find routes
travelers.find_routes(network.network_g, network.station_id_nm_dict)
travelers.set_initial_status(network.station_id_nm_dict)

display(travelers.travelers_df)

,origin_nid,destin_nid,traveler_id,departure_time,traveler_status,update_time,association,next_station,origin_station,destination_station
0,226,257,0,26871,pretrip,0,None,None,all-12TH,all-ORIN
1,229,231,1,29491,pretrip,0,None,None,all-24TH,all-ASHB
2,229,256,2,26830,pretrip,0,None,None,all-24TH,all-NCON
3,265,226,3,28823,pretrip,0,None,None,all-SANL,all-12TH
4,235,262,4,27817,pretrip,0,None,None,all-CAST,all-POWL
...,...,...,...,...,...,...,...,...,...,...
94,247,233,92,27717,pretrip,0,None,None,all-GLEN,all-BAYF
95,274,250,93,28441,pretrip,0,None,None,all-WOAK,all-LAKE
96,231,241,94,26810,pretrip,0,None,None,all-ASHB,all-DBRK
98,261,244,95,27961,pretrip,0,None,None,all-PLZA,all-EMBR


In [16]:
### check if travelers can transfer between lines
print(travelers.travelers_paths[15])
print(travelers.travelers_key_stops[15])
all_trains.schedule_df[(all_trains.schedule_df['location']=='1-COLM') & 
                      (all_trains.schedule_df['next_time']>29400+140) &
                       (all_trains.schedule_df['next_time']<29424+1000)
                      ]
all_trains.schedule_df[all_trains.schedule_df['trip_id']==964621]

['all-SANL', '12-SANL', '12-BAYF', '12-CAST', 'all-CAST']
{'all-SANL': '12-SANL', '12-SANL': '12-CAST', '12-CAST': 'all-CAST'}


,trip_id,time,next_time,status,location,current_location
16224,964621,24420.0,24450.0,stop,1-ANTC,past
16225,964621,24450.0,24840.0,on_link,1-ANTC-1-PCTR,past
16226,964621,24840.0,24870.0,stop,1-PCTR,past
16227,964621,24870.0,25200.0,on_link,1-PCTR-1-PITT,past
16228,964621,25200.0,25260.0,stop,1-PITT,past
16229,964621,25260.0,25620.0,on_link,1-PITT-1-NCON,past
16230,964621,25620.0,25650.0,stop,1-NCON,past
16231,964621,25650.0,25800.0,on_link,1-NCON-1-CONC,past
16232,964621,25800.0,25830.0,stop,1-CONC,past
16233,964621,25830.0,26100.0,on_link,1-CONC-1-PHIL,past


### Simulation and plotting

In [11]:
def plot_map(network, trains, travelers, t):
    
    fig, ax = plt.subplots(figsize=(20, 20))
    ### network
    network.all_links.plot(ax=ax, zorder=1)
    network.all_nodes.plot(ax=ax, zorder=2)
    ### train positions
    train_positions = trains.get_all_train_positions(network)
    trains_on_link = train_positions[train_positions['status']=='on_link']
    U = trains_on_link['ux'] / np.sqrt(trains_on_link['ux']**2 + trains_on_link['uy']**2);
    V = trains_on_link['uy'] / np.sqrt(trains_on_link['ux']**2 + trains_on_link['uy']**2);
    ax.quiver(trains_on_link['cx'], trains_on_link['cy'],
              U, V, scale=0.1, scale_units='dots', color='red')
    trains_on_platform = train_positions[train_positions['status']=='stop']
    ax.scatter(trains_on_platform['cx'], trains_on_platform['cy'],
               facecolor="None", edgecolor='red', marker='o', s=100, zorder=3)

    ### traveler positions
    traveler_locations = travelers.get_all_traveler_positions(train_positions)
    ### annotate trip id
    traveler_locations.apply(lambda x: ax.annotate(text='{}-{}'.format(x['trip_id'], x['num_travelers']),
                                               xytext=(x['cx'], x['cy'])+np.random.random(1)/30,
                                               xy=(x['cx'], x['cy']),
                                               arrowprops=dict(arrowstyle="->", color='k'),
                                               size=10), axis=1)
    
    fig.savefig('outputs/figs/t{}.png'.format(t))

In [12]:
def save_results(network, trains, travelers, t):
    ### save train results
    train_positions = trains.get_all_train_positions(network)
    train_positions.to_csv('outputs/train_outputs/train_outputs_od100_{}.csv'.format(t), index=False)
    ### save traveler results
    traveler_locations = travelers.travelers_df.groupby(
            ['traveler_status', 'association']).size().to_frame(
            name='num_travelers').reset_index(drop=False)
    traveler_locations.to_csv('outputs/traveler_outputs/traveler_outputs_od100_{}.csv'.format(t), index=False)

In [13]:
for t in range(26664-200, 30700, 20):
    ### update train location
    all_trains.update_location_occupancy(t)
    
    ### update traveler status
    travelers.traveler_update(network, all_trains, t)
    
    ### print and plot results
    #plot_map(network, all_trains, travelers, t)
    save_results(network, all_trains, travelers, t)
    